In [7]:
import pymongo
from pymongo import MongoClient, GEOSPHERE
import sys
sys.path.append('../src')
import os
import code4_data_ as f4
import geojson

ModuleNotFoundError: No module named 'geojson'

## Establishing connection to mongoDB collection that match criteria with the offices selected

In [2]:
client = MongoClient()

In [3]:
berlin = client.ironhack.hqreq_berlin
paris = client.ironhack.hqreq_paris
amsterdam = client.ironhack.hqreq_amsterdam
companies2 = client.ironhack.companies2

## Connecting to collections with locations that matches criteria

## Establishing connection to mongoDB

In [4]:
df = f4.import_csv("../data_processed/gdf5_v1.csv")

In [5]:
df

,name,category_code,city,country_code,latitude,longitude,geometry,city_coord
0,plista,advertising,Berlin,DEU,52.528361,13.412456,"{'type': 'Point', 'coordinates': (13.4124563, ...","Berlin, 10117, Germany"
1,99designs,design,Berlin,DEU,52.498620,13.446903,"{'type': 'Point', 'coordinates': (13.4469031, ...","Berlin, 10117, Germany"
2,babbel,education,Berlin,DEU,52.489700,13.388610,"{'type': 'Point', 'coordinates': (13.38861, 52...","Berlin, 10117, Germany"
3,Deezer,music,Paris,FRA,48.878060,2.328385,"{'type': 'Point', 'coordinates': (2.3283853, 4...","Paris, Ile-de-France, Metropolitan France, France"
4,Adknowledge,advertising,Paris,FRA,48.850606,2.379783,"{'type': 'Point', 'coordinates': (2.3797825, 4...","Paris, Ile-de-France, Metropolitan France, France"
5,Netsize,mobile,Paris,FRA,48.894312,2.288319,"{'type': 'Point', 'coordinates': (2.288319, 48...","Paris, Ile-de-France, Metropolitan France, France"
6,Solairedirect,cleantech,Paris,FRA,48.874595,2.321074,"{'type': 'Point', 'coordinates': (2.3210737, 4...","Paris, Ile-de-France, Metropolitan France, France"
7,2 Minutes,games_video,Paris,FRA,48.850206,2.369803,"{'type': 'Point', 'coordinates': (2.3698033, 4...","Paris, Ile-de-France, Metropolitan France, France"
8,Bomgar,software,Paris,FRA,48.875231,2.298337,"{'type': 'Point', 'coordinates': (2.2983373, 4...","Paris, Ile-de-France, Metropolitan France, France"
9,Viadeo,social,Paris,FRA,48.874706,2.330416,"{'type': 'Point', 'coordinates': (2.3304161, 4...","Paris, Ile-de-France, Metropolitan France, France"


In [8]:
import ast

In [16]:
for each in df.geometry:
    each = ast.literal_eval(each)
    each['new'] = list(each['coordinates'])
    print(each['new'])


[13.4124563, 52.5283614]
[13.4469031, 52.4986199]
[13.38861, 52.4897]
[2.3283853, 48.8780596]
[2.3797825, 48.8506065]
[2.288319, 48.8943115]
[2.3210737, 48.8745951]
[2.3698033, 48.8502056]
[2.2983373, 48.8752305]
[2.3304161, 48.8747064]
[2.324782, 48.86931]
[4.8516964, 52.3965603]
[4.870312, 52.340609]
[4.8909347, 52.3738007]


0     {'type': 'Point', 'coordinates': (13.4124563, ...
1     {'type': 'Point', 'coordinates': (13.4469031, ...
2     {'type': 'Point', 'coordinates': (13.38861, 52...
3     {'type': 'Point', 'coordinates': (2.3283853, 4...
4     {'type': 'Point', 'coordinates': (2.3797825, 4...
5     {'type': 'Point', 'coordinates': (2.288319, 48...
6     {'type': 'Point', 'coordinates': (2.3210737, 4...
7     {'type': 'Point', 'coordinates': (2.3698033, 4...
8     {'type': 'Point', 'coordinates': (2.2983373, 4...
9     {'type': 'Point', 'coordinates': (2.3304161, 4...
10    {'type': 'Point', 'coordinates': (2.324782, 48...
11    {'type': 'Point', 'coordinates': (4.8516964, 5...
12    {'type': 'Point', 'coordinates': (4.870312, 52...
13    {'type': 'Point', 'coordinates': (4.8909347, 5...
Name: geometry, dtype: object

In [10]:
berlin_offices = df.iloc[:3,6]

In [11]:
paris_offices = df.iloc[3:11,6]

In [12]:
amsterdam_offices = df.iloc[11:,6]

In [13]:
df.geometry

0     {'type': 'Point', 'coordinates': (13.4124563, ...
1     {'type': 'Point', 'coordinates': (13.4469031, ...
2     {'type': 'Point', 'coordinates': (13.38861, 52...
3     {'type': 'Point', 'coordinates': (2.3283853, 4...
4     {'type': 'Point', 'coordinates': (2.3797825, 4...
5     {'type': 'Point', 'coordinates': (2.288319, 48...
6     {'type': 'Point', 'coordinates': (2.3210737, 4...
7     {'type': 'Point', 'coordinates': (2.3698033, 4...
8     {'type': 'Point', 'coordinates': (2.2983373, 4...
9     {'type': 'Point', 'coordinates': (2.3304161, 4...
10    {'type': 'Point', 'coordinates': (2.324782, 48...
11    {'type': 'Point', 'coordinates': (4.8516964, 5...
12    {'type': 'Point', 'coordinates': (4.870312, 52...
13    {'type': 'Point', 'coordinates': (4.8909347, 5...
Name: geometry, dtype: object

In [87]:
berlin_ = []
for office_loc in berlin_offices:
    office_loc = ast.literal_eval(office_loc)
    office_loc['coordinates'] = list(office_loc['coordinates'])
    q1 = {"location": {"$near": {"$geometry": office_loc, "$maxDistance": 2000}}}
    berlin_ = list(berlin.find(q1))
    

OperationFailure: error processing query: ns=ironhack.hqreq_berlinTree: GEONEAR  field=location maxdist=2000 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error :: caused by :: unable to find index for $geoNear query, full error: {'ok': 0.0, 'errmsg': 'error processing query: ns=ironhack.hqreq_berlinTree: GEONEAR  field=location maxdist=2000 isNearSphere=0\nSort: {}\nProj: {}\n planner returned error :: caused by :: unable to find index for $geoNear query', 'code': 291, 'codeName': 'NoQueryExecutionPlans'}

In [73]:
for office_loc in berlin_offices:
    office_loc = ast.literal_eval(office_loc)
    office_loc['coordinates'] = list(office_loc['coordinates'])
    print(office_loc)
    


{'type': 'Point', 'coordinates': [13.4124563, 52.5283614]}
{'type': 'Point', 'coordinates': [13.4469031, 52.4986199]}
{'type': 'Point', 'coordinates': [13.38861, 52.4897]}


In [53]:
print(office)

{'type': 'Point', 'coordinates': (13.4124563, 52.5283614)}


In [ ]:
q1 = {"location": {"$near": {"$geometry": office_loc, "$maxDistance": 2000}}}

In [60]:
off_coord = list(office['coordinates'])

In [61]:
type(off_coord)

list

In [64]:
off_coord

[13.4124563, 52.5283614]

In [62]:
q1 = {"location": {"$near": {"$geometry": office, "$maxDistance": 2000}}}

In [63]:
berlin_ = list(berlin.find(q1))

OperationFailure: error processing query: ns=ironhack.hqreq_berlinTree: GEONEAR  field=location maxdist=2000 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error :: caused by :: unable to find index for $geoNear query, full error: {'ok': 0.0, 'errmsg': 'error processing query: ns=ironhack.hqreq_berlinTree: GEONEAR  field=location maxdist=2000 isNearSphere=0\nSort: {}\nProj: {}\n planner returned error :: caused by :: unable to find index for $geoNear query', 'code': 291, 'codeName': 'NoQueryExecutionPlans'}

In [47]:
for office_loc in berlin_offices:
    office_loc = dict(office_loc)
    print(office_loc)
    print(type(office_loc))
    q1 = {"location": {"$near": {"$geometry": dict(office_loc), "$maxDistance": 2000}}}
    berlin_ = list(berlin.find(q1))
    print(berlin_)

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [ ]:
berlin_ = list(berlin.find(q1))